In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import os

# Get the path of the current notebook
notebook_path = '/content/drive/My Drive/COMPSCI602'

# Change the current working directory to the notebook's directory
os.chdir(notebook_path)

print(f"Current working directory: {os.getcwd()}")


Current working directory: /content/drive/My Drive/COMPSCI602


In [3]:
%pip install git+https://github.com/TattaBio/DGEB.git
%pip install esm
import dgeb

  Cloning https://github.com/TattaBio/DGEB.git to /tmp/pip-req-build-z3y31htx
  Running command git clone --filter=blob:none --quiet https://github.com/TattaBio/DGEB.git /tmp/pip-req-build-z3y31htx
  Resolved https://github.com/TattaBio/DGEB.git to commit 1b187e607e278a34ba7338b8b43747c57add4134
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## ESM Model Evaluation with Custom Metrics

We'll evaluate ESM models on operon prediction using DGEB's standard pipeline, but adding ROC-AUC metric.

In [4]:
from huggingface_hub import login
login()

In [5]:
### APPROACH 1: Monkey-Patch (Run this cell to use Approach 1)

# Monkey-patch DGEB to add ROC-AUC metric (simplest approach!)
import add_roc_auc_to_dgeb  # This patches PairClassificationEvaluator

# Now import DGEB normally
import dgeb
from dgeb.tasks import EcoliOperon

print("✅ Approach 1: DGEB loaded with ROC-AUC metric added!")

✅ PairClassificationEvaluator patched to include ROC-AUC metric
   Now all DGEB tasks using this evaluator will report ROC-AUC!
✅ Approach 1: DGEB loaded with ROC-AUC metric added!


In [8]:
### APPROACH 1: Run Evaluation (Run this after Approach 1 setup)

model_name = "esm3_sm_open_v1"
# Check if ESM3 and set environment variable
if "esm3" in model_name.lower():
    os.environ['ESM3_DEFAULT_DTYPE'] = 'float32'

# Load model
print(f"Loading model: {model_name}...")
model = dgeb.get_model(model_name)

# Convert to float32 after loading
if "esm3" in model_name.lower():
    model.encoder = model.encoder.float()  # or model.encoder



# Use DGEB's standard way - EcoliOperon task will now include ROC-AUC!
print("Running evaluation with standard EcoliOperon task...")
evaluation = dgeb.DGEB(tasks=[EcoliOperon])

# Run evaluation (writes results to results/ folder)
# Add num_workers to avoid the warning
results = evaluation.run(model, output_folder="results/approach1", num_workers=12)

Loading model: esm3_sm_open_v1...
✅ Model loaded with batch_size=1
Running evaluation with standard EcoliOperon task...


───────────────────────────────────────────────── Selected Tasks  ─────────────────────────────────────────────────

- E.coli Operonic Pair, pair_classification

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
encoding: 100%|██████████| 4315/4315 [11:25<00:00,  6.30it/s]



✅ Evaluation Complete!


Display results

In [14]:
import pandas as pd

# Display results in a nice DataFrame format
# Extract scores from DGEB TaskResult format
# DGEB returns results per layer, we'll use the best performing layer
print(f"\n📊 ESM Model Performance on Operon Prediction")
print(f"Model: {results[0].model.hf_name}")
print(f"Layers evaluated: {len(results[0].results)}\n")

# Show results for each layer
for layer_result in results[0].results:
    layer_num = layer_result.layer_number
    print(f"\n{'='*70}")
    print(f"Layer {layer_num} Results")
    print('='*70)

    # Convert TaskMetric objects to dictionary
    layer_scores = {m.id: m.value for m in layer_result.metrics}

    # Organize by distance metric
    layer_results_dict = {}
    for metric_type in ["cos_sim", "euclidean", "manhattan", "dot"]:
        layer_results_dict[metric_type] = {}
        for metric in ["accuracy", "precision", "recall", "f1", "ap", "roc_auc"]:
            key = f"{metric_type}_{metric}"
            if key in layer_scores:
                layer_results_dict[metric_type][metric] = layer_scores[key]

    # Create DataFrame
    layer_df = pd.DataFrame(layer_results_dict).T
    layer_df = layer_df.round(4)

    # Calculate and add the average row
    average_row = layer_df.mean()
    layer_df.loc['Average'] = average_row.round(4)

    display(layer_df)

    # Highlight best metric
    # Exclude the 'Average' row when finding the best metric
    best_distance = layer_df.drop('Average').loc[:, 'f1'].idxmax()
    print(f"\n🏆 Best distance metric (excluding average): {best_distance}")
    print(f"   F1 Score: {layer_df.loc[best_distance, 'f1']:.4f}")
    # Safely access roc_auc and ap, providing a default if not present
    roc_auc_score = layer_df.loc[best_distance, 'roc_auc'] if 'roc_auc' in layer_df.columns else "N/A"
    ap_score = layer_df.loc[best_distance, 'ap'] if 'ap' in layer_df.columns else "N/A"
    print(f"   ROC-AUC:  {roc_auc_score:.4f}" if isinstance(roc_auc_score, float) else f"   ROC-AUC:  {roc_auc_score}")
    print(f"   AP Score: {ap_score:.4f}" if isinstance(ap_score, float) else f"   AP Score:  {ap_score}")


📊 ESM Model Performance on Operon Prediction
Model: esm3_sm_open_v1
Layers evaluated: 2


Layer 24 Results


,accuracy,precision,recall,f1,ap,roc_auc
cos_sim,0.6351,0.4519,0.8477,0.5896,0.5472,0.6382
euclidean,0.6254,0.4274,0.9582,0.5911,0.5259,0.6134
manhattan,0.6338,0.4369,0.8998,0.5882,0.5418,0.6317
dot,0.5948,0.4051,1.0000,0.5766,0.3845,0.4867
Average,0.6223,0.4303,0.9264,0.5864,0.4998,0.5925



🏆 Best distance metric (excluding average): euclidean
   F1 Score: 0.5911
   ROC-AUC:  0.6134
   AP Score: 0.5259

Layer 47 Results


,accuracy,precision,recall,f1,ap,roc_auc
cos_sim,0.6210,0.4242,0.9416,0.5849,0.5137,0.6052
euclidean,0.6456,0.4533,0.8867,0.5999,0.5681,0.6643
manhattan,0.6458,0.4681,0.8689,0.6084,0.5700,0.6653
dot,0.5967,0.4049,0.9994,0.5763,0.3725,0.4298
Average,0.6273,0.4376,0.9242,0.5924,0.5061,0.5911



🏆 Best distance metric (excluding average): manhattan
   F1 Score: 0.6084
   ROC-AUC:  0.6653
   AP Score: 0.5700


In [ ]:
layer_df

,accuracy,precision,recall,f1,ap,roc_auc
cos_sim,0.6210,0.4242,0.9416,0.5849,0.5137,0.6052
euclidean,0.6456,0.4533,0.8867,0.5999,0.5681,0.6643
manhattan,0.6458,0.4681,0.8689,0.6084,0.5700,0.6653
dot,0.5967,0.4049,0.9994,0.5763,0.3725,0.4298
Average,0.6273,0.4376,0.9242,0.5924,0.5061,0.5911
